## Importaciones

In [66]:
import pandas as pd
import math
import numpy as np
import csv
import hashlib
import os
import re

from tqdm import tqdm

## Cargar datos y sacar muestra

In [67]:
req_cols = ['id','screen_name','text']
row_num = math.ceil(0.2 * 4594980)  # 20% de los datos

# Abrir el archivo CSV y procesarlo línea por línea
tweets_df = pd.read_csv("tweets_2022_abril_junio.csv", usecols=req_cols, nrows=row_num)

tweets_df.columns

Index(['id', 'screen_name', 'text'], dtype='object')

In [68]:
len(tweets_df)

918996

In [69]:
for index, row in tqdm(tweets_df.iterrows()):
    text = row['text'].replace('\n', '').replace('\r', '')
    text = re.sub(r'^RT\s+@\w+:\s+', '',text).lower()
    tweets_df.at[index, 'text'] = text

918996it [01:13, 12468.96it/s]


In [70]:
tweets_df.head()

,id,screen_name,text
0,1512186166438637582,h0l4d4ni3l4,"tras casi 50 años del golpe, la constitución s..."
1,1512186202367045642,Claudio70932894,mañana jueves a las 18hrs. comienza nuestro pr...
2,1512186287284924418,Cesar_A_RR,aquí está el aporte de @tere_marinovic con res...
3,1512186335754301446,rosmarieher,la pelotudez no tiene limites...no tiene
4,1512186407841767424,GQuelluen,"ante la circulación de noticias falsas, les qu..."


## Obtener Shingles por tweet y todos los shingles

In [71]:
k = 3 ## Largo de los shingles
tweets_df["shingles"] = [set([tweet[i:i+k] for i in range(len(tweet) - k + 1)]) for tweet in tqdm(tweets_df["text"])]

100%|██████████| 918996/918996 [00:30<00:00, 29994.88it/s]


In [72]:
union_shingles = set()

for row in tqdm(tweets_df['shingles']):
    # Realizar la unión con el conjunto combinado
    union_shingles = union_shingles.union(row)

100%|██████████| 918996/918996 [1:22:43<00:00, 185.17it/s]


In [73]:
union_shingles

{'=un',
 'l#…',
 '3jv',
 'no(',
 'l 😁',
 '👏😍🇵',
 '6n8',
 'tór',
 'clv',
 'q6q',
 'ez»',
 '09/',
 'bh(',
 '🤣💔 ',
 '👇🏻r',
 '3mv',
 'kz6',
 '🌿☘️',
 '🤬😡😨',
 '𝚜 ,',
 'pló',
 '🙄¿t',
 'lv ',
 '🌹💖💖',
 'n8)',
 'cap',
 '7i5',
 'j3q',
 ',0 ',
 'rqt',
 '390',
 'o#8',
 '/za',
 '😆🤡🍿',
 '8c7',
 's…h',
 '9u2',
 'kol',
 'i59',
 ' 😎👍',
 'í!m',
 'nqq',
 '🇻🇪…',
 '8si',
 ' 😂…',
 '0we',
 'n 🐍',
 '!!🦾',
 'xn4',
 '_u ',
 'yzz',
 'k2z',
 ' *1',
 'o9q',
 'ox4',
 '🤣😋😋',
 'i4r',
 'ub4',
 ' 😄🤡',
 '.😳n',
 '.ta',
 ' 🙄🙃',
 'e 3',
 '😅🤦🏻',
 '😱es',
 'bon',
 '🤢 i',
 'bng',
 '56/',
 '👆de',
 'ojf',
 'usj',
 'cnq',
 'wdl',
 'cl✨',
 '‼️👹',
 ' 🙄b',
 'cbt',
 'geo',
 '4-1',
 '💰 n',
 '1!!',
 '😜 h',
 '5cz',
 '@ze',
 'ov6',
 'vk3',
 'hjt',
 'lkx',
 '2ua',
 '️🥴 ',
 'a!😡',
 'náo',
 ' 🤦🤭',
 'w3c',
 '_ z',
 '8g1',
 'zgw',
 'xly',
 '¡ep',
 'zbj',
 '3zf',
 '4q2',
 '✊💜 ',
 'qtq',
 'nwi',
 'ccb',
 'e👉5',
 '@ub',
 ' 6🟡',
 'í h',
 '“su',
 '2yy',
 '❓❓n',
 'ile',
 'r 8',
 'os5',
 '😄😅🍄',
 '3gp',
 '/ "',
 'bid',
 '!🌱ú',
 'áig',
 '8jr',
 'l18',


## Similitud de Jaccard

In [74]:
def jaccard_similarity(set1, set2):
    # Computa la similitud de Jaccard entre dos sets
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    return len(intersection) / len(union)

## Funciones de Hash

In [75]:
def crear_hash(salt):
    return hashlib.sha1(salt)

In [76]:
h = []
num_hash = 100
for i in range(num_hash):
  salt = os.urandom(32)
  h.append(crear_hash(salt))

## Almacenamos los shingles para ahorrar tiempo

In [77]:
ruta = 'all_shingles' + str(k) + '.txt'
with open(ruta,'w', encoding="utf-8") as file:
	for item in union_shingles:
		file.write(item+"\n")

## Cargamos los shingles

In [78]:
union_shingles = []
with open(ruta, 'r', encoding="utf-8") as f:
  [union_shingles.append(line) for line in f.readlines()]
del union_shingles[-1]

In [79]:
print(len(union_shingles))

113680


## Calculamos el minhash de los tweets

In [80]:
sorted_union = sorted(union_shingles)
union_lista = list(union_shingles)

### con esto podemos usar names[i] para conseguir el nombre del i-esimo tweet. 
names = tweets_df["id"]

In [81]:
### Creamos la matriz FH con las filas y columnas correspondientes. 
### Inicialmente la matriz tiene solo valores 3000, cualquier valor mas grande 
### que la cantidad de elementos sirve. 

n = len(names)
FH = np.full((num_hash, n), np.inf)

### iteramos cada elemento
for e in tqdm(range(0,len(sorted_union))):
    ### iteramos cada columna, correspondiente a la cantidad de textos (odas en nuestro caso) 
    for i in range(0,n):
        ### si el elemento e no está en el texto i, no hacemos nada
        if sorted_union[e].replace('\n', '') not in tweets_df["shingles"][i]:
          pass
        ### si el elemento si está, computamos los valores de h(e) y 
        else:
          for j in range(0,num_hash):
            h[j].update(bin(e).encode('utf-8'))
            FH[j][i] = min(FH[j][i],int(h[j].hexdigest(), 16))

  4%|▍         | 4460/113680 [6:22:22<156:03:56,  5.14s/it] 


KeyboardInterrupt: 

In [ ]:
union_lista[40]

' có\n'

In [ ]:
print(len(FH))
print(len(FH[0]))

100
460


In [ ]:
FH

array([[2.54237225e+46, 8.14291399e+45, 7.56984088e+45, ...,
        9.09711117e+44, 2.37259840e+46, 1.05473557e+46],
       [3.66486243e+45, 2.61930126e+46, 1.37149068e+45, ...,
        2.44178115e+45, 6.05575322e+45, 4.29324690e+45],
       [5.33776404e+45, 6.29761762e+45, 8.75478569e+45, ...,
        9.99587812e+45, 8.50335309e+45, 5.97934689e+45],
       ...,
       [1.26858065e+46, 1.14693405e+46, 7.56008169e+45, ...,
        3.20219870e+46, 1.90903695e+45, 1.28448442e+46],
       [9.59049367e+45, 3.57822608e+45, 1.50477699e+46, ...,
        4.17640914e+45, 3.60930543e+46, 1.03850828e+46],
       [3.30789663e+46, 2.25032985e+46, 5.26949574e+44, ...,
        1.27451752e+45, 3.78546237e+46, 5.64176829e+45]])

## Almacenamos los minhash para ahorrar tiempo

In [ ]:
np.savetxt('minhash.txt', FH)